In [1]:
import numpy as np
import pickle
import os
from config import config

#function from https://www.cs.toronto.edu/~kriz/cifar.html
def unpickle(file):
    with open(file, 'rb') as f:
        dict = pickle.load(f, encoding='bytes')
    return dict

In [2]:
cifarpath = config['paths']['cifardatapath']
test_folder = config['dataset_details']['test_set']
train_data = []
train_labels = []
if os.path.exists(cifarpath):
    if len(os.listdir(cifarpath))>0:
        for x in os.listdir(cifarpath):
            if 'data' in x and x != test_folder:
                current_data = unpickle(os.path.join(cifarpath,x))
                train_data.extend(current_data[b'data'])
                train_labels.extend(current_data[b'labels'])
else:
    print("Please download the data first")
train_data = np.array(train_data)
train_labels = np.array(train_labels)
test_batch = unpickle(os.path.join(cifarpath, test_folder))
test_data = np.array(test_batch[b'data'])
test_labels = np.array(test_batch[b'labels'])

In [3]:
#analyzing batches meta
meta_batches = unpickle(os.path.join(cifarpath, 'batches.meta'))
class_names =[x.decode('utf-8') for x in meta_batches[b'label_names']] 

#mapping between int and string for recognition of labels
#labels in the train/test are stored as integers and in batches meta as strings
mapping_classnames = dict()
for i,value in enumerate(class_names):
    mapping_classnames[i] = value

In [25]:
#now restrict the dataset 

#first - analysis of label distribution in the training set
import pandas as pd
import random

percent_of_dataset = 0.1
train_labels_pd = pd.Series(train_labels)

#ok, we see that the distribution is even, so we can take 10% from each label
restricted_train_data = []
restricted_train_labels = []
for label in list(mapping_classnames.keys()):
    indices_of_label = np.where(train_labels == label)[0]
    #now take percent_of_dataset random indices
    number_to_take = int(len(indices_of_label)*percent_of_dataset)
    random.seed(42)
    random.shuffle(indices_of_label)
#     random_subset = np.random.choice(indices_of_label, size=number_to_take, replace=False)
    random_subset = indices_of_label[:number_to_take]
    restricted_train_data.extend(train_data[random_subset])
    restricted_train_labels.extend(train_labels[random_subset])

print("Before {} items. Now {} items".format(len(train_data),len(restricted_train_data)))
#check whether it is indeed 500 for each 
# for label in list(mapping_classnames.keys()):
#     print(len(np.where(np.array(restricted_train_labels) == label)[0]))


Before 50000 items. Now 5000 items


In [26]:
import pickle
path_to_restricted_dataset = config['paths']['restricted_dataset_path']
restricted_data_dict = {}
restricted_data_dict["data"] = restricted_train_data
restricted_data_dict["labels"] = restricted_train_labels
with open(path_to_restricted_dataset, 'wb') as f: 
    pickle.dump(restricted_data_dict, f)

In [27]:
#save to file


['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
